# Assignment by Ishika Gupta

In [1]:
# Importing necesary libraries
import pandas as pd
import psycopg2 #PostgreSQL database adapter for Python

In [2]:
# Establishing connection to render postgresql database and accessing "ratings" table
conn = psycopg2.connect(
host="dxx-xxxxxxxxxxxx-x.oregon-postgres.render.com",
database="db_name",
user="username",
password="*****************************"
)


cur = conn.cursor()
cur.execute("SELECT * FROM movies")

# Fetch the results into a DataFrame
columns = [desc[0] for desc in cur.description]
df_movies = pd.DataFrame(cur.fetchall(), columns=columns)

conn.commit()
cur.close()
conn.close()

In [3]:
# Establishing connection to render postgresql database and accessing "ratings" table
conn = psycopg2.connect(
host="dxx-xxxxxxxxxxxx-x.oregon-postgres.render.com",
database="db_name",
user="username",
password="*****************************"
)


cur = conn.cursor()
cur.execute("SELECT * FROM ratings")

# Fetch the results into a DataFrame
columns = [desc[0] for desc in cur.description]
df_ratings = pd.DataFrame(cur.fetchall(), columns=columns)

conn.commit()
cur.close()
conn.close()

In [4]:
# Storing movies table in "movies"
movies = pd.DataFrame(df_movies)

In [5]:
# Storing ratings table in "ratings"
ratings = pd.DataFrame(df_ratings)

In [6]:
#Dimensions of "movies" table
movies.shape

(3143, 8)

In [7]:
#Dimensions of "ratings" table
ratings.shape

(10000, 4)

In [8]:
#Display the top 5 rows for movies table
movies.head()

,id,title,year,country,genre,director,minutes,poster
0,3112654,Mea culpa,2014,France,"Action, Thriller",Fred Cavayé,90,http://ia.media-imdb.com/images/M/MV5BNTQ0ODAw...
1,1571222,A Dangerous Method,2011,"UK, Germany, Canada, Switzerland","Biography, Drama, Thriller",David Cronenberg,99,http://ia.media-imdb.com/images/M/MV5BMTU5Mjk3...
2,2756032,The One I Love,2014,USA,"Comedy, Drama, Romance",Charlie McDowell,91,http://ia.media-imdb.com/images/M/MV5BMTgzODMy...
3,2370248,Short Term 12,2013,USA,Drama,Destin Daniel Cretton,96,http://ia.media-imdb.com/images/M/MV5BMTEwNjE2...
4,57171,"Yesterday, Today and Tomorrow",1963,"Italy, France","Comedy, Romance",Vittorio De Sica,119,http://ia.media-imdb.com/images/M/MV5BMTY0NDc4...


In [9]:
#Display the top 5 rows for ratings table
ratings.head()

,rater_id,movie_id,rating,time
0,1,68646,10,1381620027
1,1,113277,10,1379466669
2,2,422720,8,1412178746
3,2,454876,8,1394818630
4,2,790636,7,1389963947


In [10]:
#Information about movies table and its attributes
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3143 non-null   int64 
 1   title     3143 non-null   object
 2   year      3143 non-null   int64 
 3   country   3143 non-null   object
 4   genre     3143 non-null   object
 5   director  3143 non-null   object
 6   minutes   3143 non-null   int64 
 7   poster    3143 non-null   object
dtypes: int64(3), object(5)
memory usage: 196.6+ KB


In [11]:
#Information about ratings table and its attributes
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   rater_id  10000 non-null  int64
 1   movie_id  10000 non-null  int64
 2   rating    10000 non-null  int64
 3   time      10000 non-null  int64
dtypes: int64(4)
memory usage: 312.6 KB


In [12]:
#Checking total number of null values for each column of movies table
movies.isna().sum()

id          0
title       0
year        0
country     0
genre       0
director    0
minutes     0
poster      0
dtype: int64

In [13]:
#Checking total number of null values for each column of ratings table
ratings.isna().sum()

rater_id    0
movie_id    0
rating      0
time        0
dtype: int64

In [14]:
#Changing name of "id" column in "movies" table to "movie_id" for merging
movies.rename({'id':'movie_id'}, axis='columns', inplace=True)

In [15]:
# Merging movies and ratings tables on the basis of "movie_id" column
df = movies.merge(ratings,on="movie_id")

In [16]:
# Dropping unnecessary columns
df.drop(['poster','time'],axis=1,inplace=True)

In [17]:
# Creating new columns
# Create 'num_ratings_per_movie' representing the number of times each movie is rated
df['num_ratings_per_movie'] = df.groupby('title')['rating'].transform('count')

# Create 'num_movies_rated_by_rater' representing the count of movies rated by each rater
df['num_movies_rated_by_rater'] = df.groupby('rater_id')['rating'].transform('count')

In [18]:
#Checking the dimensions of merged table
df.shape

(10000, 11)

In [19]:
# Replace all "N/A" with python null values
df.replace("N/A", pd.NA, inplace=True)

In [20]:
# checking number of null values in new dataframe
df.isna().sum()

movie_id                     0
title                        0
year                         0
country                      2
genre                        4
director                     1
minutes                      0
rater_id                     0
rating                       0
num_ratings_per_movie        0
num_movies_rated_by_rater    0
dtype: int64

In [21]:
# Since the number of null values is much lesser than the cardinality of data, we can drop them
df.dropna(inplace=True)

# Answer 1
a. Top 5 Movie Titles: Sort and print the top 5 movie titles based on the following criteria:
● Duration
● Year of Release
● Average rating (consider movies with minimum 5 ratings)
● Number of ratings given

## By duration

In [22]:
#Sorting values in dataframe by duration in ascending order and storing in "df1_a"
df1_a = df.sort_values(by=['minutes']).head()

In [23]:
#Title of top 5 movies with lowest duration in ascending order of movie title
df1_a['title'].sort_values().values.tolist()

['After My Garden Grows',
 'Lekar Hum Deewana Dil',
 'Romantik Komedi 2: Bekarliga Veda',
 'Romeo Ranjha',
 'Sonsuz']

## By year of release

In [24]:
#Sorting values in dataframe by year of release in ascending order and storing in "df1_b"
df1_b = df.sort_values(by=['year']).head()

In [25]:
#Title of top 5 movies with earliest release date in ascending order of movie title
df1_b['title'].sort_values().values.tolist()

['Behind the Screen',
 'Hot Water',
 'Häxan: Witchcraft Through the Ages',
 'The Gold Rush',
 'The Navigator']

## By average rating

In [26]:
#Taking only the rows where movies have at least 5 ratings and storing the new dataframe in "df1"
df1 = df[df['num_ratings_per_movie']>=5]

Note: df1 is the dataframe with movies with at least 5 ratings

In [27]:
#Sorting values in dataframe by average rating (movies with more than 5 ratings) in ascending order and storing in "df1_c"
df1_c = df1.sort_values(by=['rating']).head()

In [28]:
#Title of top 5 movies with lowest average rating in ascending order of movie title
df1_c['title'].sort_values().values.tolist()

['A Most Wanted Man',
 'Brick Mansions',
 'Into the Woods',
 'Requiem for a Dream',
 'The Thin Red Line']

## By number of ratings

In [29]:
#Sorting values in dataframe by number of ratings per movie in ascending order and storing in "df1_d"
df1_d = df.sort_values(by=['num_ratings_per_movie']).head()

In [30]:
#Title of top 5 movies with lowest number of ratings in ascending order of movie title
df1_d['title'].sort_values().values.tolist()

['A Dark Truth', 'Drive Hard', 'Earth to Echo', 'Mea culpa', 'Muscle Shoals']

# Answer 2
b. Number of Unique Raters: Determine and print the count of unique rater IDs

In [31]:
# Number of unique rater IDs
df['rater_id'].nunique()

1047

# Answer 3
c. Top 5 Rater IDs: Sort and print the top 5 rater IDs based on:
● Most movies rated
● Highest Average rating given (consider raters with min 5 ratings)

## By number of movies rated

In [32]:
# Sorting data frame by number of ratings by each rater (grouping by rater_id) 
# in descending order and storing the new dataframe in "df3_a"
df3_a = df.sort_values(by=['num_movies_rated_by_rater'],ascending=False)

In [33]:
# Top 5 rater IDs who have rated maximum number of movies in ascending order
top_5_raters_id = df3_a['rater_id'].value_counts().head(5).index.tolist()
top_5_raters_id.sort()
print(top_5_raters_id)

[82, 512, 735, 769, 800]


## By Highest Average rating given (consider raters with min 5 ratings)

In [34]:
# Filtering out records with raters who have rated at least 5 movies and storing in "df3"
df3 = df[df['num_movies_rated_by_rater']>=5]

Note: df3 is the dataframe with raters who have done at least 5 ratings

In [35]:
# Sorting values in dataframe by average rating (movies with more than 5 ratings) by 
# each rater (grouping by rater_id) in descending order and storing in "df3_b"
df3_b = df3.sort_values(by=['rating'],ascending=False)

In [36]:
# extracting raters id in descending order of their number of ratings
top_raters_id = df3_b['rater_id'].head().values.tolist()

In [37]:
# Top 5 raters_id who have rated maximum number of movies in ascending order
top5_raters_id = top_raters_id[:5]
top5_raters_id.sort()
print(top5_raters_id)

[88, 314, 318, 485, 678]


# Answer 4
d. Top Rated Movie:
- Find and print the top-rated movies by:
● Director 'Michael Bay',
● 'Comedy',
● In the year 2013
● In India (consider movies with a minimum of 5 ratings)

### Assumption - Considering the movies with at least 7 rating score as "top-rated"

## By Director 'Michael Bay'

In [38]:
# Sorting dataframe by movies with director "Michael Bay" having at least 7 rating score and storing it in df4_a
df4_a = df[(df['director']=='Michael Bay') & (df['rating']>=7)].sort_values(by=['rating'], ascending=False)

In [39]:
# Top rated 5 movies by Director "Michael Bay" in ascending order
top_rated_5_movies = df4_a['title'].value_counts().head(5).index.tolist()
top_rated_5_movies.sort()
print(top_rated_5_movies)

['Pain & Gain', 'Transformers', 'Transformers: Age of Extinction', 'Transformers: Dark of the Moon', 'Transformers: Revenge of the Fallen']


## By Comedy

In [40]:
# Sorting dataframe by movies with genre "Comedy" having at least 7 ratings and storing it in df4_b
df4_b = df[(df['genre']=='Comedy') & (df['rating']>=7)].sort_values(by=['rating'], ascending=False)

In [41]:
#Top-rated 5 movies in genre "Comedy"
df4_b['title'].head().values.tolist()

["Ferris Bueller's Day Off",
 'Grown Ups 2',
 'Mac & Devin Go to High School',
 'In a World...',
 'Authors Anonymous']

## By year as 2013

In [42]:
# Sorting dataframe by movies with year of release "2013" having at least 7 ratings and storing it in df4_c
df4_c = df[(df['year']==2013) & (df['rating']>=7)].sort_values(by=['rating'], ascending=False)

In [43]:
#Top-rated 5 movies in the year 2013
df4_c['title'].head().values.tolist()

['12 Years a Slave',
 'The Wolf of Wall Street',
 'The Wolverine',
 'Riddick',
 'Dallas Buyers Club']

## By country as India (movies with a minimum of 5 ratings)

Note: df1 is the dataframe with movies with at least 5 ratings

In [44]:
# Sorting dataframe by movies with country as "India" having at least 7 ratings and storing it in df4_d
df4_d = df1[(df1['country']=='India') & (df1['rating']>=7)].sort_values(by=['rating'], ascending=False)

In [45]:
#Top-rated movies in India in ascending order
top_rated_movies_india = df4_d['title'].value_counts().head(5).index.tolist()
top_rated_movies_india.sort()
print(top_rated_movies_india)

['3 Idiots', 'PK']


## Answer 5
e. Favorite Movie Genre of Rater ID 1040: Determine and print the favorite movie genre
for the rater with ID 1040 (defined as the genre of the movie the rater has rated most often).

In [46]:
# Filtering records having ratings for rater id "1040" only
df5 = df[df['rater_id']==1040]

In [47]:
# Create new column 'Number_of_time_genre_rated' representing number of times each genre is rated by raters
df5['Number_of_time_genre_rated'] = df5.groupby('genre')['rating'].transform('count')

<ipython-input-47-245880cc46e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Number_of_time_genre_rated'] = df5.groupby('genre')['rating'].transform('count')


In [48]:
# Sorting records in descending order by number of times each genre is rated
df5.sort_values(by=['Number_of_time_genre_rated'],ascending=False)

,movie_id,title,year,country,genre,director,minutes,rater_id,rating,num_ratings_per_movie,num_movies_rated_by_rater,Number_of_time_genre_rated
1321,1631867,Edge of Tomorrow,2014,"USA, Canada","Action, Adventure, Sci-Fi",Doug Liman,113,1040,9,37,113,11
1033,1300854,Iron Man 3,2013,"USA, China","Action, Adventure, Sci-Fi",Shane Black,130,1040,7,40,113,11
6954,796366,Star Trek,2009,"USA, Germany","Action, Adventure, Sci-Fi",J.J. Abrams,127,1040,10,6,113,11
7059,1408101,Star Trek Into Darkness,2013,USA,"Action, Adventure, Sci-Fi",J.J. Abrams,132,1040,10,42,113,11
3358,1430132,The Wolverine,2013,"USA, UK, Australia, Japan","Action, Adventure, Sci-Fi",James Mangold,126,1040,6,32,113,11
...,...,...,...,...,...,...,...,...,...,...,...,...
4422,3622592,Paper Towns,2015,USA,"Drama, Mystery, Romance",Jake Schreier,109,1040,7,4,113,1
7586,2361184,Tapped Out,2014,Canada,"Action, Drama, Sport",Allan Ungar,105,1040,4,1,113,1
3758,2243537,A Haunted House,2013,USA,"Comedy, Horror",Michael Tiddes,86,1040,3,3,113,1
5614,2112124,Chennai Express,2013,India,"Action, Comedy, Romance",Rohit Shetty,141,1040,10,1,113,1


In [49]:
# Find the index where a genre is rated maximum number of times
max_genre_index = df5['Number_of_time_genre_rated'].idxmax()

# Get the corresponding genre value for the maximum number of times a genre is rated
max_genre_value = df5.loc[max_genre_index, 'genre']

# Display the result
print("Favourite Genre for rater with ID 1040:", max_genre_value)

Favourite Genre for rater with ID 1040: Action, Adventure, Sci-Fi


# Answer 6
f. Highest Average Rating for a Movie Genre by Rater ID 1040: Find and print the
highest average rating for a movie genre given by the rater with ID 1040 (consider genres with a
minimum of 5 ratings).

Note: df5 is the dataframe with details of rater with ID 1040 and the number of times he rated each genre

In [50]:
# Filtering records having ratings for rater id "1040" only and sorting them in descending order by number of ratings
df6 = df5[df5['Number_of_time_genre_rated']>=5].sort_values(by=['rating'],ascending=False)

In [51]:
# Find the index where the rating is maximum
max_rating_index = df6['rating'].idxmax()

# Get the corresponding genre and rating value for the maximum rating
max_rated_genre_value = df6.loc[max_rating_index, 'genre']
highest_avg_rating = df6.loc[max_rating_index,'rating']

# Display the result
print("Highest average rating is {} for genre {} by rater with ID 1040.".format(highest_avg_rating, max_rated_genre_value))

Highest average rating is 10 for genre Action, Adventure, Sci-Fi by rater with ID 1040.


# Answer 7
g. Year with Second-Highest Number of Action Movies: Identify and print the year with
the second-highest number of action movies from the USA that received an average rating of
6.5 or higher and had a runtime of less than 120 minutes.

In [52]:
# Sorting data frame for movies where country is USA, average rating>=6.5,
# runtime of less than 120 minutes and genre as Action and storing in df7
df7 = df[(df['country']=='USA') & (df['rating']>=6.5) & (df['minutes']<120) & (df['genre']=='Action')]

In [53]:
# Calculate movie count per year
movie_count_per_year = df7["year"].value_counts()

# Find year with second-highest count
if len(movie_count_per_year)>1:
    second_highest_year = movie_count_per_year.nlargest(2).index[1]
else:
    second_highest_year = movie_count_per_year.index[0]

# Print the year
print(f"Year with second-highest number of matching action movies: {second_highest_year}")

Year with second-highest number of matching action movies: 2003


# Answer 8
h. Count of Movies with High Ratings: Count and print the number of movies that have
received at least five reviews with a rating of 7 or higher.

Note: df1 is the dataframe with movies with at least 5 ratings

In [54]:
# Filtering out records where rating for movie is 7 or higher and storing in df8
df8 = df1[df1['rating']>=7]

In [55]:
# Displaying the result
print("Total number of movies that have received at least five reviews with a rating of 7 or higher is {}.".format(df8['title'].nunique()))

Total number of movies that have received at least five reviews with a rating of 7 or higher is 459.
